# Аббревиатуры 

- ОМС - Обязательное медицинское страхование Подробнее: http://thedifference.ru/chem-otlichaetsya-dms-ot-oms/
- ДМС - Добровольное медицинское страхование Подробнее: http://thedifference.ru/chem-otlichaetsya-dms-ot-oms/
- СОМП - случай оказываемой медицинской помощи
- КСГ - клиника статистическая группа (стационар - услуги, операции, но мы ему присваиваем группу)
- СМО - страховая медицинская организация

# Введение

В настоящее время объемы любых данных, будь то корпоративная база данных, хранящая факты о клиентах или продажах, настолько массивны, что для человека это просто не по силам - проанализировать их самостоятельно. Отсюда вытекает ряд проблем у многих компаний, занимающихся разработкой и внедрением программного обеспечения в учреждения здравоохранения. Сотрудникам организаций, обеспечивающих медицинские учреждения, приходится проверять огромные объемы информации. 
 


Одним из решений может стать выявление ассоциативных правил в базе данных.  Рассмотрим алгоритм Apriori, который находит закономерности между связанными данными.

В данной работе выявим правила, некие знания, в базе данных, содержащей медицинские карточки клиентов, проходящих обслуживание в поликлиниках. Тем самым мы сможем в дальнейшем определять, допущена ли ошибка в заполнении той или иной карте.


# Постановка задачи

В данной работе рассмотрим следующие задачи:
    1. Адаптировать алгоритм нахождения ассоциативных правил в данных, полученных заполнением медицинских карточек.
    2. Применить полученные правила для выявления и анализа зависимостей между теми или иными данными.
    3. Рассмотреть возможное применение обнаруженных правил для дальнейшего контроля заполнения мед. карточек

# 1. Алгоритм нахождения ассоциативных данных

Под термином "транзакция" мы будем понимать данные о пациенте, внесенные врачом в его карту: возраст, пол, диагноз и т.д. Пример такой транзакции: sex = Male, medic = Врач-хирург, ... Количественные данные, такие как возраст, будем обозначать интервалом: age = 14-18.

Тем самым мы получим некое количество транзакций, число которых равно числу пациентов. 

Попробуем выявить в них знания, уделяя особое внимание тем полям, в которых часто допускаются ошибки при заполнении.

### 1.1 Рассмотрим ошибки, связанные с несоответствием должности специалиста и характеристик пациента(возраста, пола и диагноза) 

*Данный набор ошибок взят из cправочника ["Причины возврата счетов" SPR15](http://www.kubanoms.ru/infirmac_obmen1.html) *


* 351 - значение в поле "Пол" не соответствует диагнозу
* 361 - мужчине в гинекологии оказаны медицинские услуги
* 362 - в урологии оказаны медицинские услуги: Мужчине с диагнозами N70-N98, N99.2, N99.3 или Женщине с диагнозами N40-N51
* 507 - профиль ОМП (или специальность врача) не соответствует диагнозу
* 508 - профиль ОМП (или специальность врача) не соответсвуют возрасту пациента
* 861 - оказанная медицинская услуга не соответствует полу пациента
* 875 - условие оказания поликлинической МП или медицинская услуга не соответствует возрасту пациента
* 911 - несоответствие профиля оказанной МП и специальности медицинского сотрудника
* 931 - Диагноз не соответствует специальности




**Постановка задачи:** для проверки такого рода ошибок составим правило Х -> У такое, чтобы Х(характеристики пациента) соотвествовали врачу У.

**Обработка данных:** сделаем выборку, состящую из: пола(1 - М, 2 - Ж), возраста, диагноза пациента; специальности медицинского работника.

In [1]:
import mysql.connector

In [18]:
db = mysql.connector.connect(user='root', password='12345', host='127.0.0.1', database='newdata')
query = db.cursor()
query.execute("""SELECT DISTINCT  client.sex,
     TIMESTAMPDIFF(year, client.birthDate, action.createDatetime) as age, 
     mkb.BlockName, 
     rbpost.name
    FROM diagnosis 
    JOIN mkb ON diagnosis.mkb = mkb.diagid
    JOIN client ON diagnosis.client_id = client.id
    JOIN event ON event.client_id = client.id
    JOIN action ON action.event_id = event.id
    JOIN person ON action.person_id = person.id
    JOIN rbpost ON person.post_id = rbpost.id LIMIT 10
""")
for i in query.fetchall():
    print(i)
db.close()

(2, 47, 'Обращение в учреждения здравоохранения для медицинского осмотра и обследования', 'Врач-терапевт участковый')
(2, 47, 'Потенциальная опасность для здоровья, связанная с инфекционными болезнями', 'Врач-терапевт участковый')
(2, 70, 'Обращение в учреждения здравоохранения для медицинского осмотра и обследования', 'Врач-терапевт участковый')
(1, 21, 'Болезни пищевода, желудка и двенадцатиперстной кишки', 'Врач-терапевт участковый')
(1, 54, 'Болезни наружного уха', 'Врач-невролог')
(1, 54, 'Болезни полости рта, слюнных желез и челюстей', 'Врач-невролог')
(1, 54, 'Болезни пищевода, желудка и двенадцатиперстной кишки', 'Врач-невролог')
(1, 54, 'Мочекаменная болезнь', 'Врач-невролог')
(1, 54, 'Болезни мышц глаза, нарушения содружественного движения глаз, аккомодации и рефракции', 'Врач-невролог')
(1, 54, 'Болезни наружного уха', 'Врач-отоларинголог')


Создадим выборку из 100 тыс. таких записей в карточках. Запишем ее в *csv* файл для дальнейшего применения на ней алгоритма [*Apriori*](https://en.wikipedia.org/wiki/Apriori_algorithm)

In [9]:
import csv

wf = open('Problem_1.csv', 'w', newline='\n')
writer = csv.writer(wf, delimiter=';', quotechar='|')
    
def write(a):
    writer.writerow(a)

In [23]:
import mysql.connector

db = mysql.connector.connect(user='root', password='12345', host='127.0.0.1', database='newdata')
query = db.cursor()
query.execute("""SELECT DISTINCT  client.sex,
     TIMESTAMPDIFF(year, client.birthDate, action.createDatetime) as age, 
     mkb.BlockName, 
     rbpost.name
    FROM diagnosis 
    JOIN mkb ON diagnosis.mkb = mkb.diagid
    JOIN client ON diagnosis.client_id = client.id
    JOIN event ON event.client_id = client.id
    JOIN action ON action.event_id = event.id
    JOIN person ON action.person_id = person.id
    JOIN rbpost ON person.post_id = rbpost.id LIMIT 100000
""")
for i in query.fetchall():
    write(i)
    
    
wf.close()
db.close()

Записав выборку в Problem_1.csv, перейдем к поиску правил. Для этого нам надо преобразовать данные в базу транзакций. Воспользуемся пакетом [arules](https://cran.r-project.org/web/packages/arules/index.html) языка R.
* Cчитаем базу из файла, бинаризуем данные и построим правила, используя альгоритм apriori с поддержкой равной 0,00001 и достоверностью - 0,1. 


* У нас получилось 50647 правил. Выделим из них те, которые слева содержат характеристики, а справа специальности врачей. Для этого зададим параметру rhs(right-hand-side) ту характеристику(DOCTOR), которую хотим видеть справа. 

* В итоге получим 6348 правил, подходящих под наши требования

Запишем выборку только для Врача-хирурга, тем самым увелечим значение параметра *support* 

In [6]:
import csv

wf = open('Problem_obstetrician.csv', 'w', newline='\n')
writer = csv.writer(wf, delimiter=';', quotechar='|')
    
def write(a):
    writer.writerow(a)

In [7]:
import mysql.connector

db = mysql.connector.connect(user='root', password='12345', host='127.0.0.1', database='newdata')
query = db.cursor()
query.execute("""select DISTINCT  client.sex,
     TIMESTAMPDIFF(year, client.birthDate, diagnostic.createDatetime) as age, 
     mkb.BlockName, 
     rbpost.name
    from diagnostic
    join person on diagnostic.person_id = person.id
    join rbpost on person.post_id = rbpost.id
    join diagnosis on diagnostic.diagnosis_id = diagnosis.id
    join client on diagnosis.client_id = client.id
    join mkb on diagnosis.mkb = mkb.diagid
    where rbpost.name = 'Акушерка'
""")
for i in query.fetchall():
    write(i)
    
    
wf.close()
db.close()


*Хирург*
- itemFrequencyPlot - s=0.1
- rules - s=0.0001

*Акушерка*
- пол и диагноз 
- специальность и диагноз 
- специальность и возраст 
- диагноз и возраст (supp = 0.00001, conf = 0.001,)
- специальность и пол (supp = 0.00001, conf = 0.01)

Пол и специальность: 

Диагноз и возраст ([*полная версия*](https://public.tableau.com/profile/publish/Diagnosisandage/Sheet1#!/publish-confirm))

### 1.2 Ошибки, связанные с несоответствием возраста пациента и проведенной для него процедуры

* 894 - электрокардиография не проведена (либо проведена не по возрасту)
* 895 - не проведена флюорография (с возраста 15 лет) (либо проведена не по возрасту)
* 896 - не проведена нейросонография (детям первого года жизни) (либо проведена не по возрасту)

* 898 - не проведено УЗИ органов брюшной полости, сердца
* 899 - не проведено УЗИ тазобедренных суставов (детям первого года жизни) (либо проведена не по возрасту)

**Данные:** для проверки такого рода ошибок составим правило Х -> У такое, чтобы Х(характеристики пациента) соотвествовали врачу У. Для этого сделаем выборку возраста пациента и услуги, которые ему прописали. 

In [4]:
import csv

wf = open('Problem_2.csv', 'w', newline='\n')
writer = csv.writer(wf, delimiter=';', quotechar='|')

def write(a):
    b = []
    b.append(a[0])
    if a[1] == 'Прием (осмотр, консультация) врача ■ травматолога-ортопеда первичный':
        b.append('Прием (осмотр, консультация) врача травматолога-ортопеда первичный')
    elif a[1] == 'Прием (осмотр, консультация) врача ■ травматолога-ортопеда повторный':
        b.append('Прием (осмотр, консультация) врача травматолога-ортопеда повторный')
    else: 
        b.append(a[1])
    writer.writerow(b)

In [ ]:
import mysql.connector

db = mysql.connector.connect(user='root', password='12345', host='127.0.0.1', database='newdata')
query = db.cursor()
query.execute("""SELECT DISTINCT TIMESTAMPDIFF(year, client.birthDate, event.createDatetime) as age,
    actiontype.name as service
    FROM client 
    JOIN event ON client.id = event.client_id
    JOIN action ON action.event_id = event.id
    JOIN actiontype ON action.actionType_id = actiontype.id LIMIT 10000
""")

#for i in query.fetchall():
#    write(i)
    
    
#wf.close()
db.close()

Далее проделаем все то же, что и в пункте 1.

*Для выявления большего количества "знаний" приходится брать поддержку как можно маньше.*

* Применим ItemFrequencyPlot с support = 0.01 и получим распределение 

* В итоге получим 1513 правил(support = 0.0001). Используем приложение [Tableau Public](https://public.tableau.com/s/) для удобной визуализации решения. 

Рассмотрим еще один вид графика, построенного в [Tableau Public](https://public.tableau.com/s/):

* 897 - не проведено УЗИ щетовидной железы и органов репродуктивной сферы (с возраста 7 лет) (либо проведена не по возрасту)

### 1.3

In [1]:
import csv

wf = open('Problem_whole.csv', 'w', newline='\n')
writer = csv.writer(wf, delimiter=';', quotechar='|')
    
def write(a):
    writer.writerow(a)

In [2]:
import mysql.connector

db = mysql.connector.connect(user='root', password='12345', host='127.0.0.1', database='newdata')
query = db.cursor()
query.execute("""select  
   rbeventtypepurpose.name as rbeventtypepurpose_name, 
   rbfinance.name as rbfinance_name,
   rbmedicalaidkind.name as rbmedicalaidkind_name, 
   rbmedicalaidtype.name as rbmedicalaidtype_name,
   TIMESTAMPDIFF(year, client.birthDate, event.createDatetime) as client_age,
   client.sex as client_sex,
   rbpost.name as execperson_name,
   rbspeciality.name as rbspeciality_name,
   rbdocumenttype.name as rbdocumenttype_name,
   visit.isPrimary as visit_isPrimary,
   action.status as action_status,
   contract.regionalTariffRegulationFactor as contract_regionalTariffRegulationFactor
from event
join eventtype on event.eventType_id = eventtype.id
join rbeventtypepurpose on eventtype.purpose_id = rbeventtypepurpose.id
join rbfinance on eventtype.finance_id = rbfinance.id
join rbmedicalaidkind on eventtype.medicalAidKind_id = rbmedicalaidkind.id
join rbmedicalaidtype on eventtype.medicalAidtype_id = rbmedicalaidtype.id
join client on event.client_id = client.id
join person on event.execPerson_id = person.id
join rbpost on person.post_id = rbpost.id
join rbspeciality on person.speciality_id = rbspeciality.id
join clientdocument on  client.id = clientdocument.client_id
join rbdocumenttype on clientdocument.documentType_id = rbdocumenttype.id
join visit on visit.event_id = event.id
join action on event.id = action.event_id
join contract on action.contract_id = contract.id limit 100000
""")
for i in query.fetchall():
    write(i)
    
    
wf.close()
db.close()

Преобразуем наши карточки в удобный для дальнейшей работы вид. (Позже надо реализовать добавление в карту номера action или event)


In [37]:
import csv

fieldnames = ['id','rbeventtypepurpose_name','rbfinance_name','rbmedicalaidkind_name','rbmedicalaidtype_name',
              'client_age','client_sex','execperson_name',
              'rbspeciality_name','rbdocumenttype_name']

list_of_cards = []
one_card = dict.fromkeys(fieldnames)
count = -1
with open('Problem_whole.csv', 'r') as csvfile:
    filereader = csv.reader(csvfile, delimiter=';', quotechar='|')
    for row in filereader:
        one_card = dict.fromkeys(fieldnames)
        if (count >= 0):
            j = 0
            for key in one_card:
                one_card[key] = row[j][1:len(row[j]) - 1]
                j += 1
            list_of_cards.append(one_card)
        count += 1
            
        

In [38]:
list_of_cards[16]

{'client_age': '60-100',
 'client_sex': 'Female',
 'execperson_name': 'Врач-терапевт',
 'id': '17',
 'rbdocumenttype_name': 'ПАСПОРТ РОССИИ',
 'rbeventtypepurpose_name': 'Дневной стационар',
 'rbfinance_name': 'ОМС',
 'rbmedicalaidkind_name': 'специализированная медицинская помощь',
 'rbmedicalaidtype_name': 'Стационар дневного пребывания взрослый',
 'rbspeciality_name': 'Терапевт'}

Подготовим наши правила:

In [39]:
class HashMap:
    def __init__(self):
        self.size = 1000
        self.map = [None] * self.size
        
    def _get_hash(self, key):
        hash = 0 
        for char in str(key):
            hash += ord(char)
        return hash % self.size
    
    def add(self, key, value):
        key_hash = self._get_hash(key)
        key_value = [key, value]
        
        if self.map[key_hash] is None:
            self.map[key_hash] = list([key_value])
            return True
        else:
            for pair in self.map[key_hash]:
                if pair[0] == key:
                    pair[1] += '*' + value
                    return True
            self.map[key_hash].append(key_value)
            return True
        
    def get(self, key):
        key_hash = self._get_hash(key)
        if self.map[key_hash] is not None:
            for pair in self.map[key_hash]:
                if pair[0] == key:
                    return pair[1]
        return None
    
    def delete(self, key):
        key_hash = self._get_hash(key)
        if self.map[key_hash] is None:
            return False
        for i in range (0, len(self.map[key_hash])):
            if self.map[key_hash][i][0] == key:
                self.map[key_hash].pop(i)
                return True
    
    def print(self):
        for item in self.map:
            if item is not None:
                print(str(item))
                

In [49]:
import csv

list_of_rules = HashMap()
left_part_of_rules = []
count = -1

with open('result_09_08.csv', 'r') as csvfile:
    filereader = csv.reader(csvfile, delimiter=';', quotechar='|')
    for row in filereader:
        if (count >= 0):
            
            lhs = row[1].index('=>', 0, len(row[1])) 
            rule_lhs = row[1][2:lhs - 2]
            rule_rhs = row[1][lhs + 4:len(row[1]) - 2]
            
            if (rule_lhs != '') and (rule_rhs != ''):
                
                equal = rule_rhs.find('=', 0, len(rule_rhs))
                rule_lhs += '*' + rule_rhs[0: equal]
                rule_rhs = rule_rhs[equal + 1:]
                
                left_part_of_rules.append(rule_lhs)
                list_of_rules.add(rule_lhs, rule_rhs)
        count += 1

Проверка

In [51]:
wf = open('bug_report.csv', 'w', newline='\n')
writer = csv.writer(wf, delimiter=';', quotechar='|')

count = 0

for i in range(0, len(list_of_cards)):
    card = list_of_cards[i]
    rules = []
    rules.append(list_of_cards[i])
    rules.append('')
    
    mistake = False
    
    for j in range(0, len(left_part_of_rules)):
        error = False;
        lhs = left_part_of_rules[j]
        rule = lhs
    
        #left side
        while (lhs.find('=', 0, len(lhs)) != -1) and (error == False):
            
            equal = lhs.find('=', 0, len(lhs))
            check_key = str(lhs[0: equal])
            
            end = lhs.find(",", 0, len(lhs))
            if end == -1:
                end = lhs.find('*', 0, len(lhs))
            check_value = str(lhs[equal + 1: end])
    
            if card.get(check_key) != check_value: 
                error = True # наличие несоответствия левой части правила и карточки
            
            lhs = lhs[end + 1:]
            
        #right side
        if (error == False):
            rhs = list_of_rules.get(rule)
            right_card = False

            if rhs.find('*', 0, len(rhs)) == -1:
                if card.get(lhs) == rhs: 
                    right_card = True # в этом месте в карте ошибка не допущена
            else:         
                while (rhs.find('*', 0, len(rhs)) != -1):
    
                    star = rhs.find('*', 0, len(rhs))
        
                    k = str(rhs[0: star])
                    
                    if card.get(k) == rhs:
                        right_card = True
                    rhs = rhs[star + 1:]
            
            if right_card == False: # ошибка допущена
                mistake = True
                mis = [rule, list_of_rules.get(rule)]
                rules.append(mis)
    writer.writerow(rules)
    if (mistake == True): 
        count += 1
wf.close()   
print(count * 100 / (len(list_of_cards) + 1))

3.053435114503817


|                          | Эксперимент 1       | Эксперимент 2     |
| ------------------------ |---------------------| ------------------|
| support, %               | 80                  |     90            |
| confidence, %            | 80                  |     80            |
| неправильный карточки, % | 18,914334181509755  |3.053435114503817  |